# Week 3 assignment 
# Explore and cluster the neighborhoods in Toronto 
## Part 1
### 1. Import necessary library


In [26]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen

### 2. scrap the webpage

In [11]:
quote_page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 
page = urlopen(quote_page).read()  
soup = BeautifulSoup(page, 'html.parser')

table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [12]:
df.shape

(180, 3)

Initial table has 180 rows.

### 3. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.


In [13]:
df=df[df['Borough']!="Not assigned"]
df.shape

(103, 3)

We removed 77 cells that has unassigned borough

### 4. More than one neighborhood can exist in one postal code area.

In [14]:
df['Postal Code'].is_unique

True

We checked that all Postal code is unique, hence the rows with same postal code has already been combined

### 5. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [15]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df[df['Neighbourhood']=='Not assigned"'].shape

(0, 3)

We checked that there are no cells with unassigned neighbourhood

### 6.Print the number of rows of your dataframe.

In [16]:
df.shape

(103, 3)

We has 103 rows in the dataframe now

# Part 2
### 1. Import the geospatial data from csv file

In [17]:
geodata=pd.read_csv("http://cocl.us/Geospatial_data")
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### 2. Merge two data frame

In [18]:
df2=df.merge(geodata, left_on='Postal Code', right_on='Postal Code')
df2.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Part 3
### 1. Set up the environment

In [8]:
import json # library to handle JSON files 

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                       

### 2. Work with only the borough that contains Toronto

In [22]:
df3 = df2.copy()
df3 = df3[df3['Borough'].str.contains("Toronto")]
df3.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


### 3. Analyze Each Neighborhood using Foursquare API 

In [43]:
CLIENT_ID = 'ST4O01JM33Z0IGCYAXNREQKWZYKF4AD4UMV0NBO4II4AUQ2T' 
CLIENT_SECRET = 'RZQ3S4HAHXB4VOXWE4GAYKWZ11IAFOHRGGT5QOUSQZO4ZE3K' 
VERSION = '20180605' # Foursquare API version
LIMIT = 100 
radius = 500

In [72]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes): 
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items'] 
        # return only relevant information for each nearby venue
        venues_list.append([(name,lat,lng, v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results]) 
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category'] 
    return(nearby_venues)

In [128]:
trt_venues = getNearbyVenues(names=df3['Neighbourhood'],latitudes=df3['Latitude'],longitudes=df3['Longitude'])
trt_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


#### Let's print each neighborhood along with the top 5 most common venues



In [73]:
trt_onehot = pd.get_dummies(trt_venues[['Venue Category']], prefix="", prefix_sep="") 
trt_onehot['Neighborhood'] = trt_venues['Neighborhood']   
trt_grouped = trt_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in trt_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = trt_grouped[trt_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.11
1          Restaurant  0.04
2                Café  0.04
3  Seafood Restaurant  0.04
4              Bakery  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1          Bakery  0.08
2  Breakfast Spot  0.08
3     Coffee Shop  0.08
4   Grocery Store  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.12
1           Yoga Studio  0.06
2         Auto Workshop  0.06
3  Fast Food Restaurant  0.06
4        Farmers Market  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0     Airport Service  0.13
1    Airport Terminal  0.13
2         Coffee Shop  0.07
3    Sculpture Garden  0.07
4  Airport Food Court  0.07


----Central Bay Street----
                 venue  

#### Let's put that into a pandas dataframe

In [123]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = trt_grouped['Neighborhood']

for ind in np.arange(trt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(trt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Restaurant,Cocktail Bar,Beer Bar
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Breakfast Spot,Coffee Shop,Yoga Studio
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Auto Workshop,Gym / Fitness Center,Garden Center
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Coffee Shop,Boutique,Rental Car Location
4,Central Bay Street,Coffee Shop,Sandwich Place,Japanese Restaurant,Café,Italian Restaurant


### 4. Cluster Neighborhoods

In [146]:
# to prepare the dataframe for Kmean, we need to change the key fields into numeric 
top_venues = neighborhoods_venues_sorted[['Neighborhood','1st Most Common Venue']]
top_venues['count'] = 1
top_venues=top_venues.pivot(index='Neighborhood', columns='1st Most Common Venue',values='count')
top_venues=top_venues.fillna(0)
top_venues.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


1st Most Common Venue,Airport Service,Bakery,Bar,Café,Clothing Store,Coffee Shop,Gift Shop,Greek Restaurant,Grocery Store,Health Food Store,Home Service,Light Rail Station,Mexican Restaurant,Park,Pharmacy,Restaurant,Sandwich Place
Neighborhood,,,,,,,,,,,,,,,,,
Berczy Park,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Central Bay Street,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Run k-means to cluster the neighborhood into 4 clusters based on the top venues in the neighborhood

In [148]:
# set number of clusters
kclusters = 4

trt_grouped_clustering = top_venues.drop(top_venues.columns[0], axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(trt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#### Combine the top venues with the latitude/longitude for each neighborhood 

In [149]:
merged = df3.merge(neighborhoods_venues_sorted, left_on='Neighbourhood', right_on='Neighborhood')
merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,2,"Regent Park, Harbourfront",Coffee Shop,Bakery,Park,Pub,Breakfast Spot
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,2,"Queen's Park, Ontario Provincial Government",Coffee Shop,Diner,Yoga Studio,Hobby Shop,Japanese Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,0,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Italian Restaurant,Cosmetics Shop
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,1,St. James Town,Café,Coffee Shop,Clothing Store,Restaurant,Cocktail Bar
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,0,The Beaches,Health Food Store,Pub,Trail,Dog Run,Department Store


### 5. Visualize the result

In [151]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)
# add markers to map on each neighborhood
for lat, lng, Neighbourhood in zip(df3['Latitude'], df3['Longitude'], df3['Neighbourhood']):
    label = Neighbourhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
#toronto_map

In [158]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = ['red', 'green', 'blue', 'black'] 

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors_array[cluster-1]).add_to(map_clusters)
       
map_clusters